In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
emo = pd.read_csv('EmoWordNet.txt', encoding = 'utf-8', delimiter = ';', header = 0, dtype = str)

with open('sentimentScores_final.pickle', 'rb') as f:
    sent = pickle.load(f)

In [3]:
emo.head()

,Lemma#PoS,AFRAID,AMUSED,ANGRY,ANNOYED,DONT_CARE,HAPPY,INSPIRED,SAD
0,0#n,0.087696832,0.092489284,0.154123673,0.077187786,0.136462352,0.287110508,0.084151587,0.080777978
1,1#n,0.129417618,0.108453686,0.114673931,0.114567889,0.136146642,0.161293504,0.114729221,0.120717508
2,10#n,0.13055789,0.109739453,0.115716327,0.109559403,0.151769147,0.140957108,0.111358933,0.130341739
3,100#n,0.156990048,0.126364695,0.102937362,0.087751332,0.121394833,0.109794927,0.118774226,0.175992576
4,1000#n,0.133132657,0.186631367,0.09667537,0.060371283,0.051485705,0.151349978,0.120990281,0.199363358


In [4]:
emo['PoS'] = np.nan

In [5]:
for i in range(len(emo)):
    word = emo['Lemma#PoS'][i]
    emo['Lemma#PoS'][i] = word.split('#')[0]
    emo['PoS'][i] = word.split('#')[1]

<ipython-input-5-e4a677d29ddb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emo['Lemma#PoS'][i] = word.split('#')[0]
<ipython-input-5-e4a677d29ddb>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emo['PoS'][i] = word.split('#')[1]
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [6]:
emo = emo.rename(columns = {'Lemma#PoS': 'Lemma'})

In [7]:
emo.head()

,Lemma,AFRAID,AMUSED,ANGRY,ANNOYED,DONT_CARE,HAPPY,INSPIRED,SAD,PoS
0,0,0.087696832,0.092489284,0.154123673,0.077187786,0.136462352,0.287110508,0.084151587,0.080777978,n
1,1,0.129417618,0.108453686,0.114673931,0.114567889,0.136146642,0.161293504,0.114729221,0.120717508,n
2,10,0.13055789,0.109739453,0.115716327,0.109559403,0.151769147,0.140957108,0.111358933,0.130341739,n
3,100,0.156990048,0.126364695,0.102937362,0.087751332,0.121394833,0.109794927,0.118774226,0.175992576,n
4,1000,0.133132657,0.186631367,0.09667537,0.060371283,0.051485705,0.151349978,0.120990281,0.199363358,n


In [8]:
sent.head()

,HWN_Offset,SWN_Offset,Part_of_Speech,Armenian,English,Positive,Negative,Objective
0,00014490-a,00014490,,միանգամայն բավարար,"rich,plentiful,plenteous,copious,ample",0.125000,0.000000,0.87500
1,00014490-a,00014490,,լիառատ,"rich,plentiful,plenteous,copious,ample",0.125000,0.000000,0.87500
2,"00024996-a,00818008-a,01640850-a,01687167-a,,,,","00024996,00818008,01640850,01687167,00821208,0...",,նոր,"new,young,new,novel,new,fresh",0.171875,0.046875,0.78125
3,00025470-a,00025470,,թթվային,acid,0.000000,0.375000,0.62500
4,"00029933-a,","00029933,00011160",,ագահ,"prehensile,greedy,grasping,grabby,covetous,ava...",0.000000,0.000000,1.00000


In [9]:
#create the necessary columns
columns = ['HWN_Offset', 'SWN_Offset', 'Part_of_Speech', 'Armenian', 'English', 'Positive', 'Negative', 'Objective', 'Afraid', 
           'Amused', 'Angry', 'Annoyed', 'Dont_Care', 'Happy', 'Inspired', 'Sad']
lexicon = pd.DataFrame(columns = columns)

In [10]:
new = []
for i in range(len(sent)):
    words = sent['English'][i].split(',')
    n = len(words)
    scores = np.array([0, 0, 0, 0, 0, 0, 0, 0])
    for j in range(n):
        if words[j].strip() in emo['Lemma'].values:
            ind = np.where(emo['Lemma'].values == words[j].strip())[0]
            for k in range(len(ind)):
                if emo['PoS'][ind[k]] == sent['Part_of_Speech'][i]:
                    scores = scores + np.array([float(emo['AFRAID'][ind[k]]), float(emo['AMUSED'][ind[k]]), float(emo['ANGRY'][ind[k]]), 
                                                float(emo['ANNOYED'][ind[k]]), float(emo['DONT_CARE'][ind[k]]), float(emo['HAPPY'][ind[k]]), 
                                                float(emo['INSPIRED'][ind[k]]), float(emo['SAD'][ind[k]])])
    score = scores / n
    new.append({'HWN_Offset': sent['HWN_Offset'][i], 'SWN_Offset': sent['SWN_Offset'][i], 'Part_of_Speech': sent['Part_of_Speech'][i], 'Armenian': sent['Armenian'][i],
                'English': sent['English'][i], 'Positive': sent['Positive'][i], 'Negative': sent['Negative'][i], 
                'Objective': sent['Objective'][i], 'Afraid': score[0], 'Amused': score[1], 'Angry': score[2], 'Annoyed': score[3],
                'Dont_Care': score[4], 'Happy': score[5], 'Inspired': score[6], 'Sad': score[7]})
lexicon = lexicon.append(new, ignore_index = True)
lexicon.head()

,HWN_Offset,SWN_Offset,Part_of_Speech,Armenian,English,Positive,Negative,Objective,Afraid,Amused,Angry,Annoyed,Dont_Care,Happy,Inspired,Sad
0,00014490-a,00014490,,միանգամայն բավարար,"rich,plentiful,plenteous,copious,ample",0.125000,0.000000,0.87500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00014490-a,00014490,,լիառատ,"rich,plentiful,plenteous,copious,ample",0.125000,0.000000,0.87500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"00024996-a,00818008-a,01640850-a,01687167-a,,,,","00024996,00818008,01640850,01687167,00821208,0...",,նոր,"new,young,new,novel,new,fresh",0.171875,0.046875,0.78125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,00025470-a,00025470,,թթվային,acid,0.000000,0.375000,0.62500,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"00029933-a,","00029933,00011160",,ագահ,"prehensile,greedy,grasping,grabby,covetous,ava...",0.000000,0.000000,1.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
len(lexicon)

9292

In [12]:
with open('emotionScores.pickle', 'wb') as f:
    pickle.dump(lexicon, f)
    
#save the dataframe as a text file (for those interested to read the results)
with open('emotionScores.txt', 'w', encoding = 'utf-8') as f:
    f.write(lexicon.to_string())